### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
school_data_complete.head(3)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
tot_num_schools = school_data_complete["School ID"].nunique()
tot_num_students = school_data_complete["Student ID"].count()
tot_budget = sum(school_data_complete["budget"].unique())
avg_math_score = school_data_complete["math_score"].mean()
avg_read_score = school_data_complete["reading_score"].mean()
perc_math_pass = sum(school_data_complete.math_score >= 70)/tot_num_students
perc_read_pass = sum(school_data_complete.reading_score >= 70)/tot_num_students
overall_pass = school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)]['student_name'].count()/tot_num_students

In [4]:
district_summary_df = pd.DataFrame({"Total Schools": [tot_num_schools], 
                                    "Total Students": [tot_num_students], 
                                    "Total Budget": [tot_budget], 
                                    "Average Math Score": [avg_math_score], 
                                    "Average Reading Score": [avg_read_score], 
                                    "% Passing Math": [perc_math_pass], 
                                    "% Passing Reading": [perc_read_pass], 
                                    "% Overall Passing": [overall_pass]})

In [5]:
district_summary_df = district_summary_df.style.format({"Total Budget": "${:,.2f}",
                                   "Total Students": "{:,.0f}",
                                   "Average Reading Score": "{:.2f}", 
                                   "Average Math Score": "{:.2f}", 
                                   "% Passing Math": "{:.2%}", 
                                   "% Passing Reading": "{:.2%}", 
                                   "% Overall Passing": "{:.2%}"})
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [6]:
school_summary = school_data_complete.groupby(["school_name"])

In [7]:
school_type = school_summary["type"].unique()
students_per_school = school_summary["Student ID"].count()
school_budget = school_summary["budget"].unique()
per_student_budget = school_budget / students_per_school
avg_math_school = school_summary["math_score"].mean()
avg_read_school = school_summary["reading_score"].mean()
math_pass_school = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count()/students_per_school
read_pass_school = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/students_per_school 
overall_pass_school = school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)].groupby('school_name')['Student ID'].count()/students_per_school 

In [8]:
school_summary_df = pd.DataFrame({"School Type": school_type, 
                                    "Total Students": students_per_school, 
                                    "Total School Budget": school_budget,
                                    "Per Student Budget": per_student_budget,
                                    "Average Math Score": avg_math_school, 
                                    "Average Reading Score": avg_read_school, 
                                    "% Passing Math": math_pass_school, 
                                    "% Passing Reading": read_pass_school, 
                                    "% Overall Passing": overall_pass_school})
school_summary_df.index.name = None
school_style_df = school_summary_df.copy().head(5)

In [9]:
school_style_df['School Type'] = school_style_df['School Type'].astype(str).str[1:-1]
school_style_df['School Type'] = school_style_df['School Type'].astype(str).str[1:-1]
school_style_df['Total School Budget'] = school_style_df['Total School Budget'].astype(str).str[1:-1]
school_style_df['Per Student Budget'] = school_style_df['Per Student Budget'].astype(str).str[1:-1]
school_style_df.loc[:,"Total School Budget"] = school_style_df["Total School Budget"].astype(float).map("${:,.2f}".format)
school_style_df.loc[:,"Per Student Budget"] = school_style_df["Per Student Budget"].astype(float).map("${:,.2f}".format)
school_style_df = school_style_df.style.format({"Average Math Score": "{:.2f}", 
                                                    "Average Reading Score": "{:.2f}", 
                                                    "% Passing Math": "{:.2%}", 
                                                    "% Passing Reading": "{:.2%}", 
                                                    "% Overall Passing": "{:.2%}"})


In [10]:
school_style_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%


### Another approach

In [11]:
school_data_df = school_data_complete.copy()
school_data_df["passing_math"] = school_data_df["math_score"] >= 70
school_data_df["passing_reading"] = school_data_df["reading_score"] >= 70

In [12]:
school_group = school_data_df.groupby(["school_name"]).mean()
school_group["Per Student Budget"] = school_group["budget"]/school_group["size"]
school_group["% Passing Math"] = round(school_group["passing_math"]*100,2)
school_group["% Passing Reading"] = round(school_group["passing_reading"]*100,2)
school_group["% Overall Passing"] = round(school_data_df[(school_data_df['math_score'] >= 70) & (school_data_df['reading_score'] >= 70)].groupby('school_name')['Student ID'].count()/school_group["size"]*100,2)

In [13]:
school_data_summary = pd.merge(school_group, school_data, how="left", on=["school_name", "school_name"])

In [14]:
school_summary_dataframe = pd.DataFrame({"School Name":  school_data_summary["school_name"],
                                         "School Type": school_data_summary["type"],
                                         "Total Students":school_data_summary["size_x"],
                                         "Total School Budget": school_data_summary["budget_x"],
                                         "Per Student Budget":school_data_summary["Per Student Budget"], 
                                         "Average Math Score":round(school_data_summary["math_score"],2),
                                         "Average Reading Score":round(school_data_summary["reading_score"],2), 
                                         "% Passing Math": school_data_summary["% Passing Math"],
                                         "% Passing Reading": school_data_summary["% Passing Reading"],
                                         "% Overall Passing": school_data_summary["% Overall Passing"]}) 
school_summary_df_formatted = school_summary_dataframe.copy()

In [15]:
school_summary_df_formatted["Total Students"] = school_summary_df_formatted["Total Students"].map("{:,.0f}".format)
school_summary_df_formatted["Total School Budget"] = school_summary_df_formatted["Total School Budget"].map("${:,.2f}".format)
school_summary_df_formatted["Per Student Budget"] = school_summary_df_formatted["Per Student Budget"].map("${:,.2f}".format)
#Display
school_summary_df_formatted.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [16]:
school_summary_df.sort_values(by='% Overall Passing', ascending=False, inplace=True)
top_schools = school_summary_df.copy().head(5)

In [17]:
top_schools['School Type'] = top_schools['School Type'].astype(str).str[1:-1]
top_schools['School Type'] = top_schools['School Type'].astype(str).str[1:-1]
top_schools['Total School Budget'] = top_schools['Total School Budget'].astype(str).str[1:-1]
top_schools['Per Student Budget'] = top_schools['Per Student Budget'].astype(str).str[1:-1]
top_schools.loc[:,"Total School Budget"] = top_schools["Total School Budget"].astype(float).map("${:,.2f}".format)
top_schools.loc[:,"Per Student Budget"] = top_schools["Per Student Budget"].astype(float).map("${:,.2f}".format)
top_schools = top_schools.style.format({"Average Math Score": "{:.2f}", 
                                                    "Average Reading Score": "{:.2f}", 
                                                    "% Passing Math": "{:.2%}", 
                                                    "% Passing Reading": "{:.2%}", 
                                                    "% Overall Passing": "{:.2%}"})
top_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


### Another approach

In [18]:
top_schools = school_summary_dataframe.sort_values(["% Overall Passing"], ascending=False)
top_schools.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
1,Cabrera High School,Charter,1858.0,1081356.0,582.0,83.06,83.98,94.13,97.04,91.33
12,Thomas High School,Charter,1635.0,1043130.0,638.0,83.42,83.85,93.27,97.31,90.95
4,Griffin High School,Charter,1468.0,917500.0,625.0,83.35,83.82,93.39,97.14,90.60
13,Wilson High School,Charter,2283.0,1319574.0,578.0,83.27,83.99,93.87,96.54,90.58
9,Pena High School,Charter,962.0,585858.0,609.0,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [19]:
school_summary_df.sort_values(by='% Overall Passing', ascending=True, inplace=True)
bottom_schools = school_summary_df.copy().head(5)

In [20]:
bottom_schools['School Type'] = bottom_schools['School Type'].astype(str).str[1:-1]
bottom_schools['School Type'] = bottom_schools['School Type'].astype(str).str[1:-1]
bottom_schools['Total School Budget'] = bottom_schools['Total School Budget'].astype(str).str[1:-1]
bottom_schools['Per Student Budget'] = bottom_schools['Per Student Budget'].astype(str).str[1:-1]
bottom_schools.loc[:,"Total School Budget"] = bottom_schools["Total School Budget"].astype(float).map("${:,.2f}".format)
bottom_schools.loc[:,"Per Student Budget"] = bottom_schools["Per Student Budget"].astype(float).map("${:,.2f}".format)
bottom_schools = bottom_schools.style.format({"Average Math Score": "{:.2f}", 
                                                    "Average Reading Score": "{:.2f}", 
                                                    "% Passing Math": "{:.2%}", 
                                                    "% Passing Reading": "{:.2%}", 
                                                    "% Overall Passing": "{:.2%}"})
bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


### Another approach

In [21]:
bottom_schools = school_summary_dataframe.sort_values(["% Overall Passing"], ascending=True)
bottom_schools.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
10,Rodriguez High School,District,3999.0,2547363.0,637.0,76.84,80.74,66.37,80.22,52.99
2,Figueroa High School,District,2949.0,1884411.0,639.0,76.71,81.16,65.99,80.74,53.20
7,Huang High School,District,2917.0,1910635.0,655.0,76.63,81.18,65.68,81.32,53.51
5,Hernandez High School,District,4635.0,3022020.0,652.0,77.29,80.93,66.75,80.86,53.53
8,Johnson High School,District,4761.0,3094650.0,650.0,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [22]:
nine_grade_math = school_data_complete[school_data_complete["grade"] == "9th"].groupby("school_name").mean()["math_score"]
ten_grade_math = school_data_complete[school_data_complete["grade"] == "10th"].groupby("school_name").mean()["math_score"]
eleven_grade_math = school_data_complete[school_data_complete["grade"] == "11th"].groupby("school_name").mean()["math_score"]
twelve_grade_math = school_data_complete[school_data_complete["grade"] == "12th"].groupby("school_name").mean()["math_score"]

In [23]:
math_by_school_df = round(pd.DataFrame({"9th":nine_grade_math, "10th":ten_grade_math, "11th":eleven_grade_math, "12th":twelve_grade_math}), 2)
math_by_school_df.index.name = None

In [24]:
math_by_school_df.head(5)

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [25]:
nine_grade_read = school_data_complete[school_data_complete["grade"] == "9th"].groupby("school_name").mean()["reading_score"]
ten_grade_read = school_data_complete[school_data_complete["grade"] == "10th"].groupby("school_name").mean()["reading_score"]
eleven_grade_read = school_data_complete[school_data_complete["grade"] == "11th"].groupby("school_name").mean()["reading_score"]
twelve_grade_read = school_data_complete[school_data_complete["grade"] == "12th"].groupby("school_name").mean()["reading_score"]

In [26]:
read_by_school_df = round(pd.DataFrame({"9th":nine_grade_read, "10th":ten_grade_read, "11th":eleven_grade_read, "12th":twelve_grade_read}), 2)
read_by_school_df.index.name = None

In [27]:
read_by_school_df.head(5)

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [28]:
spending_df = school_summary_dataframe.copy()
spending_df.head(2)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976.0,3124928.0,628.0,77.05,81.03,66.68,81.93,54.64
1,Cabrera High School,Charter,1858.0,1081356.0,582.0,83.06,83.98,94.13,97.04,91.33


In [29]:
spending_bins = [0, 583.99, 628.99, 643.99, 675]
spending_bin_names = ["<$584", "$585-629", "$630-644", "$645-675"]

In [30]:
spending_df["Spending Ranges (Per Student)"] = pd.cut(spending_df["Per Student Budget"],spending_bins, labels=spending_bin_names)
spending_df_grouped = spending_df.groupby("Spending Ranges (Per Student)").mean().round(2)
spending_df_grouped.drop(['Total Students', 'Total School Budget', 'Per Student Budget'], axis=1, inplace=True)
spending_df_grouped

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.45,83.94,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.99,81.92,75.21,86.09,65.71
$645-675,77.02,80.96,66.70,80.68,53.72


## Scores by School Size

* Perform the same operations as above, based on school size.

In [31]:
size_df = school_summary_dataframe.copy()
size_df["size"] = school_data_summary["size_x"]
size_df.head(2)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,size
0,Bailey High School,District,4976.0,3124928.0,628.0,77.05,81.03,66.68,81.93,54.64,4976.0
1,Cabrera High School,Charter,1858.0,1081356.0,582.0,83.06,83.98,94.13,97.04,91.33,1858.0


In [32]:
size_bins = [0, 999.99, 1999.99, 5000]
size_bin_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [33]:
size_df["School Size"] = pd.cut(size_df["size"],size_bins, labels=size_bin_names)
size_df_grouped = size_df.groupby("School Size").mean().round(2)
size_df_grouped.drop(['size', 'Total Students', 'Total School Budget', 'Per Student Budget'], axis=1, inplace=True)
size_df_grouped

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (1000-2000),83.37,83.87,93.60,96.79,90.62
Large (2000-5000),77.74,81.34,69.96,82.77,58.28


## Scores by School Type

* Perform the same operations as above, based on school type

In [34]:
type_df = school_summary_dataframe.copy()
type_df["School Type"] = school_data_summary["type"]
type_df.head(2)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976.0,3124928.0,628.0,77.05,81.03,66.68,81.93,54.64
1,Cabrera High School,Charter,1858.0,1081356.0,582.0,83.06,83.98,94.13,97.04,91.33


In [35]:
type_df_grouped = type_df.groupby("School Type").mean().round(2)
type_df_grouped.drop(['Total Students', 'Total School Budget', 'Per Student Budget'], axis=1, inplace=True)
type_df_grouped

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
